In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
ashery_chexpert_path = kagglehub.dataset_download('ashery/chexpert')

print('Data source import complete.')


100%|██████████| 10.7G/10.7G [03:14<00:00, 59.0MB/s]

Extracting files...


Data source import complete.


In [ ]:
import os

# # Iterate through the input directory without printing file paths
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     # You can perform actions with filenames here if needed
#     pass  # Replace with any desired operation on files
print(ashery_chexpert_path)

/root/.cache/kagglehub/datasets/ashery/chexpert/versions/1


In [ ]:
import pandas as pd
import numpy as np
import torch
import PIL
import matplotlib.pyplot as plt
import cv2

os.listdir(ashery_chexpert_path)

df = pd.read_csv(os.path.join(ashery_chexpert_path, 'train.csv'))
df.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [ ]:
training_data = os.path.join(ashery_chexpert_path, 'train')
patient_name = 'patient00001'
study_folder = 'study1'

image_path = os.path.join(training_data, patient_name, study_folder)
image_files = os.listdir(image_path)
print(image_path)

print("Image files found:", image_files)


if image_files:
    img_path = os.path.join(image_path, image_files[0])
    print("Loading image:", img_path)
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.axis('off')
        plt.show()
    else:
        print("Error loading image. The file may not exist or be unreadable.")
else:
    print("No images found in the specified folder.")


/root/.cache/kagglehub/datasets/ashery/chexpert/versions/1/train/patient00001/study1
Image files found: ['._view1_frontal.jpg', 'view1_frontal.jpg']
Loading image: /root/.cache/kagglehub/datasets/ashery/chexpert/versions/1/train/patient00001/study1/._view1_frontal.jpg
Error loading image. The file may not exist or be unreadable.


In [ ]:
df.fillna(0, inplace=True)

In [ ]:
base_dir = '/root/.cache/kagglehub/datasets/ashery/chexpert/versions/1/'

df['Path'] = df['Path'].str.replace('CheXpert-v1.0-small/', '', regex=False)


def get_full_image_path(relative_path):
    return f"{base_dir}{relative_path}"


df['Full_Path'] = df['Path'].apply(get_full_image_path)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223414 entries, 0 to 223413
Data columns (total 20 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Path                        223414 non-null  object 
 1   Sex                         223414 non-null  object 
 2   Age                         223414 non-null  int64  
 3   Frontal/Lateral             223414 non-null  object 
 4   AP/PA                       223414 non-null  object 
 5   No Finding                  223414 non-null  float64
 6   Enlarged Cardiomediastinum  223414 non-null  float64
 7   Cardiomegaly                223414 non-null  float64
 8   Lung Opacity                223414 non-null  float64
 9   Lung Lesion                 223414 non-null  float64
 10  Edema                       223414 non-null  float64
 11  Consolidation               223414 non-null  float64
 12  Pneumonia                   223414 non-null  float64
 13  Atelectasis   

In [ ]:

non_numeric = df[['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
                  'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation',
                  'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion',
                  'Pleural Other', 'Fracture', 'Support Devices']].applymap(lambda x: isinstance(x, (int, float)))
print(non_numeric.all().all())


<ipython-input-8-dfaf391dee3c>:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  'Pleural Other', 'Fracture', 'Support Devices']].applymap(lambda x: isinstance(x, (int, float)))


True


In [ ]:
import torch
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from PIL import Image


preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
import torch
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import cv2
import pandas as pd
import numpy as np
import timm

class XRayDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.copy()  # Create a copy to avoid modifying the original DataFrame
        self.transform = transform
        label_columns = [
            'No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion',
            'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion',
            'Pleural Other', 'Fracture', 'Support Devices'
        ]

        for col in label_columns:
            self.df[col] = pd.to_numeric(self.df[col], errors='coerce').fillna(0)

        self.df[label_columns] = self.df[label_columns].astype(np.float32)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['Full_Path']
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        if self.transform:
            image = self.transform(image)

        label_columns = [
            'No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion',
            'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion',
            'Pleural Other', 'Fracture', 'Support Devices'
        ]

        labels = self.df.iloc[idx][label_columns].to_numpy(dtype=np.float32)

        labels = np.nan_to_num(labels, nan=0.0)

        labels = torch.tensor(labels, dtype=torch.float32)

        return image, labels


In [ ]:
print(df.info())
dataset = XRayDataset(df, transform=preprocess)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

for i in range(5):  # Check the first 5 samples
    image, labels = dataset[i]
    print(f"Sample {i + 1}:")
    print("Image shape:", image.shape)
    print("Labels dtype:", labels.dtype)
    print("Labels values:", labels)
    print("-" * 20)  # Separator


# Define dataset and dataloader
print(df.info())
dataset = XRayDataset(df, transform=preprocess)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

model = timm.create_model('resnet50', pretrained=True, num_classes=0)
num_labels = df[['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion',
                 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion',
                 'Pleural Other', 'Fracture', 'Support Devices']].shape[1]

checkpoint_dir = '/content/drive/MyDrive/model_checkpoints/'
os.makedirs(checkpoint_dir, exist_ok=True)

# Check if there are any saved checkpoints
latest_checkpoint = f"{checkpoint_dir}model_epoch_5.pth"  # Adjust to your latest checkpoint
if os.path.exists(latest_checkpoint):
    model.load_state_dict(torch.load(latest_checkpoint))
    print(f"Loaded model from {latest_checkpoint}")

model.fc = nn.Linear(model.num_features, num_labels)  # Adjust final layer for multi-label classification
# ... (The rest of your code remains the same) ...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223414 entries, 0 to 223413
Data columns (total 20 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Path                        223414 non-null  object 
 1   Sex                         223414 non-null  object 
 2   Age                         223414 non-null  int64  
 3   Frontal/Lateral             223414 non-null  object 
 4   AP/PA                       223414 non-null  object 
 5   No Finding                  223414 non-null  float64
 6   Enlarged Cardiomediastinum  223414 non-null  float64
 7   Cardiomegaly                223414 non-null  float64
 8   Lung Opacity                223414 non-null  float64
 9   Lung Lesion                 223414 non-null  float64
 10  Edema                       223414 non-null  float64
 11  Consolidation               223414 non-null  float64
 12  Pneumonia                   223414 non-null  float64
 13  Atelectasis   

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

In [ ]:
# Loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # For multi-label classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# Training Loop
num_epochs = 10

checkpoint_dir = '/content/drive/MyDrive/model_checkpoints/'  # Change path as needed
os.makedirs(checkpoint_dir, exist_ok=True)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)  # Move data to the device

        # Create a mask to ignore uncertain labels (-1)
        mask = labels != -1  # Mask has True for labels that are 0 or 1

        # Set -1 labels to 0 to avoid errors in loss computation
        labels = labels.masked_fill(~mask, 0)  # Replace -1 with 0, won't affect loss due to mask

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels.float())  # Get per-element loss

        # Apply mask to ignore uncertain labels in loss
        loss = loss * mask.float()  # Zero-out loss where label is -1
        loss = loss.sum() / mask.sum()

        if (epoch + 1) % 5 == 0:
            torch.save(model.state_dict(), f"{checkpoint_dir}model_epoch_{epoch+1}.pth")
            print(f"Saved checkpoint at epoch {epoch+1}")  # Normalize by the number of valid labels

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}")

print("Training complete.")

Streaming output truncated to the last 5000 lines.
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at epoch 10
Saved checkpoint at e

In [ ]:
# I want to save the model's weights

In [ ]:
# why is this taking so long?

In [ ]:
# dont dc

In [ ]:
#yoooo